# Remote Offline Batch Inference with Ray Data & vLLM Example

This notebook presumes:
- You have a Ray Cluster URL given to you to run workloads on


In [4]:
from codeflare_sdk import RayJobClient

# Setup Authentication Configuration
auth_token = "XXXX"
header = {"Authorization": f"Bearer {auth_token}"}

In [6]:
# Gather the dashboard URL (provided by the creator of the RayCluster)
ray_dashboard = "XXXX"  # Replace with the Ray dashboard URL

# Initialize the RayJobClient
client = RayJobClient(address=ray_dashboard, headers=header, verify=True)

### Simple Example Explanation

With the RayJobClient instantiated, lets run some batch inference. The following code is stored in `simple_batch_inf.py`, and is used as the entrypoint for the RayJob.

What this processor configuration does:
- Set up a vLLM engine with your model
- Configure some settings for GPU processing
- Defines batch processing parameters (8 requests per batch, 2 GPU workers)

#### Model Source Configuration

The `model_source` parameter supports several loading methods:

* **Hugging Face Hub** (default): Use repository ID `model_source="meta-llama/Llama-2-7b-chat-hf"`
* **Local Directory**: Use file path `model_source="/path/to/my/local/model"`
* **Other Sources**: ModelScope via environment variables `VLLM_MODELSCOPE_DOWNLOADS_DIR`

For complete model support and options, see the [official vLLM documentation](https://docs.vllm.ai/en/latest/models/supported_models.html).

```python
import ray
from ray.data.llm import build_llm_processor, vLLMEngineProcessorConfig

processor_config = vLLMEngineProcessorConfig(
    model_source="replace-me",
    engine_kwargs=dict(
        enable_lora=False,
        dtype="half",
        max_model_len=1024,
    ),
    # Batch size: Larger batches increase throughput but reduce fault tolerance
    #   - Small batches (4-8): Better for fault tolerance and memory constraints
    #   - Large batches (16-32): Higher throughput, better GPU utilization
    #   - Choose based on your Ray Cluster size and memory availability
    batch_size=8,
    # Concurrency: Number of vLLM engine workers to spawn 
    #   - Set to match your total GPU count for maximum utilization
    #   - Each worker gets assigned to a GPU automatically by Ray scheduler
    #   - Can use all GPUs across head and worker nodes
    concurrency=2,
)
```

With the config defined, we can instantiate the processor. This enables batch inference by processing multiple requests through the vLLM engine, with two key steps:
- **Preprocess**: Converts each row into a structured chat format with system instructions and user queries, preparing the input for the LLM
- **Postprocess**: Extracts only the generated text from the model response, cleaning up the output

The processor defines the pipeline that will be applied to each row in the dataset, enabling efficient batch processing through Ray Data's distributed execution framework.

```python
processor = build_llm_processor(
    processor_config,
    preprocess=lambda row: dict(
        messages=[
            {
                "role": "system",
                "content": "You are a calculator. Please only output the answer "
                "of the given equation.",
            },
            {"role": "user", "content": f"{row['id']} ** 3 = ?"},
        ],
        sampling_params=dict(
            temperature=0.3,
            max_tokens=20,
            detokenize=False,
        ),
    ),
    postprocess=lambda row: {
        "resp": row["generated_text"],
    },
)
```

#### Running the Pipeline
Now we can run the batch inference pipeline on our data, it will:
- In the background, the processor will download the model into memory where vLLM serves it locally (on Ray Cluster) for use in inference
- Generate a sample Ray Dataset with 32 rows (0-31) to process
- Run the LLM processor on the dataset, triggering the preprocessing, inference, and postprocessing steps
- Execute the lazy pipeline and loads results into memory
- Iterate through all outputs and print each response 

```python
ds = ray.data.range(30)
ds = processor(ds)
ds = ds.materialize()

for out in ds.take_all():
    print(out)
    print("==========")
```

### Job Submission

Now we can submit this job against the Ray Cluster using the `RayJobClient` from earlier 

In [ ]:
import tempfile
import shutil

# Create a clean directory with ONLY your script
temp_dir = tempfile.mkdtemp()
shutil.copy("simple_batch_inf.py", temp_dir)

entrypoint_command = "python simple_batch_inf.py"

submission_id = client.submit_job(
    entrypoint=entrypoint_command,
    runtime_env={"working_dir": temp_dir, "pip": "requirements.txt"},
)

print(submission_id + " successfully submitted")

In [ ]:
# Get the job's status
client.get_job_status(submission_id)

In [ ]:
# Get the job's logs
client.get_job_logs(submission_id)